In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import os
from osgeo import gdal
import cv2

In [ ]:
def jp2_to_png(fp_in,fp_out,prefix):
    bandList = [band for band in os.listdir(fp_in) if band.endswith('.jp2')]

    for band in bandList:
        jp2_path = os.path.join(fp_in, band)  # Full path for the input JP2 file
        in_image = gdal.Open(jp2_path)
        
        if in_image is None:
            print(f"Failed to open {jp2_path}")
            continue
        driver = gdal.GetDriverByName("GTiff")
        if driver is None:
            print("GTiff driver not available")
            continue
        
        fp_tif = os.path.join(fp_in, band[:-4] + '.tif')  # Output path for the GeoTIFF
        
        if os.path.exists(fp_tif):# Check if output file already exists
            print(f"File {fp_tif} already exists, skipping.")
            continue
        out_image = driver.CreateCopy(fp_tif, in_image, 0) # Create the output image as a GeoTIFF
       
        if out_image is None:
            print(f"Failed to create {fp_tif}")

        in_image = None
        out_image = None
        
    fp_in = prefix
    band_02 = rasterio.open(fp_in + "B02_60m.tif")
    band_03 = rasterio.open(fp_in + "B03_60m.tif")
    band_04 = rasterio.open(fp_in + "B04_60m.tif")

    red = band_04.read(1)
    green = band_03.read(1)
    blue = band_02.read(1)

    rgb_composite_raw= np.dstack((red, green, blue))

    def normalize(band):
        band_min, band_max = (band.min(), band.max())
        return ((band-band_min)/((band_max - band_min)))

    red_n = normalize(red)
    green_n = normalize(green)
    blue_n = normalize(blue)

    rgb_composite_n= np.dstack((red_n, green_n, blue_n))

    def brighten(band):
        alpha=0.13
        beta=0
        return np.clip(alpha*band+beta, 0,255)

    red_b=brighten(red)
    blue_b=brighten(blue)
    green_b=brighten(green)

    red_bn = normalize(red_b)
    green_bn = normalize(green_b)
    blue_bn = normalize(blue_b)

    rgb_composite_bn= np.dstack((red_bn, green_bn, blue_bn))


    rgb_plot=plt.imshow(rgb_composite_bn, interpolation='lanczos')
    plt.axis('off')
    image_path = os.path.join(fp_out,'november.png')
    plt.savefig(image_path, dpi=200, bbox_inches='tight')
    plt.close('all')
    

prefix = "2024_november\T36UUA_20241107T090059_"
fp_out = "2024_november_png"
fp_in = "2024_november"
img_1 = jp2_to_png(fp_in,fp_out,prefix)